In [1]:
from ift6758.pipeline import ExperimentPipeline, DEFAULT_TRANSFORMATIONS

In [2]:
df = ExperimentPipeline.get_data('../data/tabular', transformations=DEFAULT_TRANSFORMATIONS)
df.head()

,game starttime,game endtime,gameId,offense_team_id,offense_team_name,offense_team_tricode,goal,x_coords,y_coords,goalie_id,...,shooter_id,shooter_name,shot type,empty net,strength_shorthand,strength_even,strength_powerplay,season,shot_angle,shot_distance
0,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,10,Toronto Maple Leafs,TOR,0,77.0,-5.0,8467950,...,8478483,Mitchell Marner,Wrist Shot,False,0.0,0.0,0.0,20162017,7.275005,13.000000
1,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,1,9,Ottawa Senators,OTT,0,86.0,13.0,8475883,...,8467967,Chris Kelly,Wrist Shot,False,0.0,0.0,0.0,20162017,2.260502,13.341664
2,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,2,9,Ottawa Senators,OTT,0,23.0,-38.0,8475883,...,8476879,Cody Ceci,Wrist Shot,False,0.0,0.0,0.0,20162017,27.460225,76.157731
3,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,3,9,Ottawa Senators,OTT,0,33.0,-15.0,8475883,...,8474578,Erik Karlsson,Slap Shot,False,0.0,0.0,0.0,20162017,28.300756,57.974132
4,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,4,10,Toronto Maple Leafs,TOR,0,34.0,-28.0,8467950,...,8475716,Martin Marincin,Wrist Shot,False,0.0,0.0,0.0,20162017,25.177535,61.717096


In [3]:
import os
from getpass import getpass

os.environ['COMET_API_KEY'] = getpass()

In [4]:
from sklearn.linear_model import LogisticRegression

pipeline = ExperimentPipeline(
    tabular_dir='../data/tabular',
    feature_columns=['shot_distance', 'shot_angle'],
    target_column='goal',
    pipeline_steps=[
        ('logreg', LogisticRegression()),
    ],
    dataset_transformations=DEFAULT_TRANSFORMATIONS,
    parameter_grid=[
        {'logreg__penalty': ['l2'], 'logreg__C': [0.1, 1, 10]},
    ],
    enable_comet=False
)

In [5]:
pipeline.run()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 3/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 4/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 5/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 1/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 3/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 4/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 5/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 1/5] END ..logreg__C=10, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END ..logreg__C=10, logreg__penalty=l2;,

In [6]:
pipeline.grid.best_score_

0.9050868966599304

In [7]:
str(pipeline.grid.best_estimator_)

"Pipeline(steps=[('logreg', LogisticRegression(C=0.1))])"